In [ ]:
from collections import namedtuple
from pathlib import Path
from pprint import pprint

from cycler import cycler
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
from scipy.stats import pearsonr, spearmanr
from more_itertools import flatten
from pyprojroot import here

import mf2
import multiLevelCoSurrogates as mlcs

np.set_printoptions(linewidth=200, edgeitems=5)

plot_dir = here() / "plots/2019-10-correlation-exploration/"
plot_dir.mkdir(parents=True, exist_ok=True)

# Plotting surfaces of Adjustable Branin and Paciorek

In [ ]:
surfaces = {
    f(0).name: list(flatten([
        mlcs.createsurfaces([f(x).high, f(x).low], l_bound=f(x).l_bound, u_bound=f(x).u_bound, step=[step, step])
        for x in [0, 0.5, 1]
    ]))
    for f, step in zip([mff.adjustable_branin, mff.adjustable_paciorek], [0.5, 0.05])
}

In [ ]:
for name, surfs in surfaces.items():
    print(name[:-1])
    mlcs.plotsurfaces(surfs, shape=(3,2), save_as=plot_dir / f'{name[:-1]}_surfaces.pdf')

# Calculating Pearson/Spearman correlations for multi-fidelity functions

In [ ]:
size_per_dim = 2000
dims = [1, 2, 3, 4, 6, 8, 10]
test_sample = {
    ndim: np.random.random((ndim*size_per_dim, ndim))
    for ndim in dims
}

Corr_result = namedtuple("Corr_result", "name ndim pearson_r pearson_r2 spearman_r spearman_r2")

In [ ]:
results = []
for f in mff.bi_fidelity_functions:
    bounds = (np.array(f.l_bound), np.array(f.u_bound))
    sample = mlcs.rescale(test_sample[f.ndim], range_in=(0,1), range_out=bounds)
    
    print(sample[:1], end='\n\n')
    
    y_h, y_l = f.high(sample), f.low(sample)
    pear, spear = pearsonr(y_h, y_l)[0], spearmanr(y_h, y_l)[0]
    results.append(Corr_result(f'{f.name}', f.ndim, pear, pear*pear, spear, spear*spear))

df = pd.DataFrame.from_records(results, columns=Corr_result._fields)
print(df)

In [ ]:
print(df.to_latex(index=False))

## A separate table for the various dimensionality options for the Forrester function

In [ ]:
results = []
for ndim in test_sample:
    bounds = (np.array(mff.forrester.l_bound), np.array(mff.forrester.u_bound))
    sample = mlcs.rescale(test_sample[ndim], range_in=(0,1), range_out=bounds)
    
    y_h, y_l = mff.forrester.high(sample), mff.forrester.low(sample)
    pear, spear = pearsonr(y_h, y_l)[0], spearmanr(y_h, y_l)[0]
    results.append(Corr_result(f'{mff.forrester.name}', ndim, pear, pear*pear, spear, spear*spear))

df = pd.DataFrame.from_records(results, columns=Corr_result._fields)
print(df)

In [ ]:
print(df.to_latex(index=False))

# Plotting Pearson/Spearman correlations for adjustable multi-fidelityfunctions

In [ ]:
params = np.linspace(0, 1, 101)
Adj_Corr_result = namedtuple("Corr_result", "name ndim param pearson_r pearson_r2 spearman_r spearman_r2")

results = []
for func in mff.adjustable_bifidelity_functions:
    for a in params:
        f = func(np.round(a, 3))
        bounds = (np.array(f.l_bound), np.array(f.u_bound))
        sample = mlcs.rescale(test_sample[f.ndim], range_in=(0,1), range_out=bounds)

        y_h, y_l = f.high(sample), f.low(sample)
        pear, spear = pearsonr(y_h, y_l)[0], spearmanr(y_h, y_l)[0]
        results.append(Adj_Corr_result(f'{func.__name__}', f.ndim, a, pear, pear*pear, spear, spear*spear))

df = pd.DataFrame.from_records(results, columns=Adj_Corr_result._fields)
df.head()

In [ ]:
plt.rcParams['axes.prop_cycle'][:4] + cycler(linestyle=['-', '--', ':', '-.'])

In [ ]:
plt.rc('axes', prop_cycle=plt.rcParams['axes.prop_cycle'][:4] + cycler(linestyle=['-', '--', ':', '-.']))

grouped_df = df.groupby('name')
for idx, (name, subdf) in enumerate(grouped_df, start=1):    
    for col in 'pearson_r pearson_r2 spearman_r spearman_r2'.split():
        plt.plot(subdf['param'], subdf[col], label=col)
    plt.axhline(y=0, color='black', alpha=.5)
    plt.xlim([0,1])
    plt.ylabel('Correlation')
    plt.xlabel(f'A{idx}')
    plt.legend(loc=0)
    plt.title(name)
    plt.tight_layout()
    plt.savefig(plot_dir / f'{name}_correlation.pdf')
    plt.show()

plt.style.use('default')